***

## Testing First LLM context generation skills

### You can use chatgpt, sagemaker llama 2, huggingface text generation interface or invoke any other endpoint

In [20]:
from openai import OpenAI

In [21]:
client = OpenAI(
    api_key=""
)

In [22]:
with open("openai-prompt", "r") as f:
    recsys_llm_prompt = f.read()

In [23]:
def get_openai_context(prompt:str, chat_history:str) -> str:
    """Get context from OpenAI model."""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": chat_history}
        ],
        temperature=1,
    )
    return response.choices[0].message.content

In [9]:
get_openai_context(recsys_llm_prompt, "pick shirts formal wear")

'I recommend the following formal shirts for you:\n- Dress shirt\n- Button-down shirt\n- Oxford shirt\n- Wingtip collar shirt\n- French cuff shirt'

In [10]:
get_openai_context(recsys_llm_prompt, "What should I wear for my brother's wedding?")

"For your brother's wedding, you can consider wearing traditional Indian attire such as a Sherwani or a Kurta-pajama. You can also opt for a bandhgala suit or a formal suit. It's a special occasion, so make sure to choose something elegant and well-tailored."

In [11]:
get_openai_context(recsys_llm_prompt, "What is the price of item no. 3?")

'##detail##'

## Testing 2nd LLM

In [13]:
import datetime
import os

import pandas as pd
import numpy as np
# import psycopg2 
from pgvector.sqlalchemy import Vector
import sshtunnel
import sqlalchemy
from sqlalchemy import Integer, String, TIMESTAMP, Float
from sqlalchemy.orm import DeclarativeBase,mapped_column,Session
from sentence_transformers import SentenceTransformer

<font color="blue">Because I was using Aurora Serverless which can only be access from inside VPC, I had to setup SSH tunnel to an ec2 instance that had access to database.</font>

In [14]:
ec2_url = ""

dbhost = ""
dbport = 1111
dbuser = ""
dbpass = ""

server = sshtunnel.SSHTunnelForwarder(
    ssh_address_or_host=(ec2_url, 22),
    ssh_username="",
    ssh_pkey="",
    remote_bind_address=(dbhost, dbport),
    local_bind_address=("localhost", 5433),
)

### I used SQLAlchemy model, you can use native psycopg2 and SQL for querying ANN

In [15]:
EMBEDDINGS_LENGTH = 768

class Base(DeclarativeBase):
    pass

class Products(Base):
    __tablename__ = "products"
    __table_args__ = {'extend_existing': True}

    pid = mapped_column(Integer, primary_key=True)
    pname = mapped_column(String)
    brand = mapped_column(String)
    gender = mapped_column(String)
    price = mapped_column(Float)
    n_images = mapped_column(Integer)
    description = mapped_column(String)
    color = mapped_column(String)
    embeddings = mapped_column(Vector(EMBEDDINGS_LENGTH))
    added_timestamp = mapped_column(TIMESTAMP)

In [16]:
model = SentenceTransformer("./embedding_model")

In [17]:
def generate_query_embeddings(user_message:str, embedding_model):
    """Generate user message embeddings."""
    openai_context = get_openai_context(recsys_llm_prompt, user_message)
    
    query_emb = embedding_model.encode(user_message + " " + openai_context)
    
    return query_emb

In [18]:
def query_product_names_from_embeddings(query_emb, engine, Table, top_k):
    """Search ANN products using embeddings."""
    with Session(engine) as session:
        stmt = sqlalchemy.select(
            Table.pid, Table.pname, Table.brand, Table.gender, Table.gender, Table.price, Table.description, Table.color
        ).order_by(Table.embeddings.l2_distance(query_emb)).limit(top_k)
        stmt_response = session.execute(stmt).mappings().all()
        
    return stmt_response

In [19]:
def get_recommendations(user_message:str, embedding_model, engine, Table, top_k=5):
    """Get recommendations."""
    embeddings = generate_query_embeddings(user_message, embedding_model)
    
    p_names = query_product_names_from_embeddings(embeddings, engine, Table, top_k)
    
    return p_names

In [24]:
server.start()

engine = sqlalchemy.create_engine(
    f"""postgresql+psycopg2://{dbuser}:"""
    f"""{dbpass}@{server.local_bind_host}:"""
    f"""{server.local_bind_port}/feature_store""",
    echo=False,
)

In [25]:
response = get_recommendations("pink strip shirts for men", model, engine, Products)

In [83]:
response

[{'pid': 10237377, 'pname': 'I AM FOR YOU Women Red & Off-White Checked Shirt Style Top', 'brand': 'I AM FOR YOU', 'gender': 'Women', 'gender_1': 'Women', 'price': 662.0, 'description': 'Red, off-white and teal blue checked woven shirt style top, has a shirt collar, long sleeves, short button placket, one pocket, curved hem', 'color': ' Red'},
 {'pid': 10197533, 'pname': 'The Pink Moon Plus Size Women White  Blue Regular Fit Striped Casual Shirt', 'brand': 'The Pink Moon', 'gender': 'Women', 'gender_1': 'Women', 'price': 1999.0, 'description': 'White and blue striped casual shirt, has a spread collar, long sleeves, button placket, and curved hem, 2 insert pockets', 'color': 'Blue'},
 {'pid': 10266737, 'pname': 'ONLY Women Red & White Regular Fit Printed Casual Shirt', 'brand': 'ONLY', 'gender': 'Women', 'gender_1': 'Women', 'price': 919.0, 'description': 'Red and white printed casual shirt, has a spread collar, long sleeves, button placket, curved hem,1 patch pocket', 'color': ' Red'},

***

In [13]:
second_llm_prompt = (
    """
    You can recommendation engine chatbot agent for an Indian apparel brand.
    You are provided with users questions and some apparel recommendations from the brand's database.
    Your job is to present the most relevant items from the data give to you.
    If user is asking a clarifying question about one of the recommended item, like what is it's price or brand, then answer that question from its description.
    Do not answer anything else apart from apparel recommendation from the company's database.
    """
)

In [5]:
def get_openai_context(prompt:str, chat_history:str) -> str:
    """Get context from OpenAI model."""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": chat_history}
        ],
        temperature=1,
    )
    return response.choices[0].message.content

In [6]:
user_query = "pink strip shirts for men"

In [7]:
recommendations = [{'pid': 10237377, 'pname': 'I AM FOR YOU Women Red & Off-White Checked Shirt Style Top', 'brand': 'I AM FOR YOU', 'gender': 'Women', 'gender_1': 'Women', 'price': 662.0, 'description': 'Red, off-white and teal blue checked woven shirt style top, has a shirt collar, long sleeves, short button placket, one pocket, curved hem', 'color': ' Red'},
 {'pid': 10197533, 'pname': 'The Pink Moon Plus Size Women White  Blue Regular Fit Striped Casual Shirt', 'brand': 'The Pink Moon', 'gender': 'Women', 'gender_1': 'Women', 'price': 1999.0, 'description': 'White and blue striped casual shirt, has a spread collar, long sleeves, button placket, and curved hem, 2 insert pockets', 'color': 'Blue'},
 {'pid': 10266737, 'pname': 'ONLY Women Red & White Regular Fit Printed Casual Shirt', 'brand': 'ONLY', 'gender': 'Women', 'gender_1': 'Women', 'price': 919.0, 'description': 'Red and white printed casual shirt, has a spread collar, long sleeves, button placket, curved hem,1 patch pocket', 'color': ' Red'},
 {'pid': 10266699, 'pname': 'ONLY Women Navy Blue & Mustard Brown Regular Fit Striped Casual Shirt', 'brand': 'ONLY', 'gender': 'Women', 'gender_1': 'Women', 'price': 1119.0, 'description': 'Navy blue and mustard brown striped casual shirt, has a spread collar, long sleeves, concealed button placket, curved hem, and 2 flap pockets', 'color': 'Blue'},
 {'pid': 10018841, 'pname': 'Raymond Men Pink Regular Fit Striped Formal Shirt', 'brand': 'Raymond', 'gender': 'Men', 'gender_1': 'Men', 'price': 1649.0, 'description': 'Pink striped formal shirt, has a spread collar, long sleeves, button placket, straight hem, and 1 patch pocket', 'color': ' Pink'}]

In [14]:
get_openai_context(second_llm_prompt, f"User question = '{user_query}', our recommendations = {recommendations}")

'Based on your query for "pink striped shirts for men", I have found a recommendation for you:\n\n- Raymond Men Pink Regular Fit Striped Formal Shirt: This pink striped formal shirt for men has a spread collar, long sleeves, button placket, straight hem, and 1 patch pocket. It is priced at Rs. 1649.0.\n\nPlease let me know if you have any further questions about this item.'

In [80]:
server.stop()
engine.dispose()

***